# Multi-Free Field Analysis Example

This example shows how to run OpenSeesMP in DesignSafe from a jupyter notebook.

A set of four 1D profiles is analyzed using OpenSeesMP. 

<img src = "multi-freeField.png"  height="400" width="400" align = "center">

# Setup Tapis and start OpenSeesMP job

In [1]:
!python -m pip install --upgrade numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 66.1 MB/s eta 0:00:00:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas 2.2.1 requires numpy<2,>=1.26.0; python_version >= "3.12", but you have numpy 2.2.6 which is incompatible.
scipy 1.11.3 requires numpy<1.28.0,>=1.21.6, but you have numpy 2.2.6 which is incompatible.


### Setup job description

In [2]:
!pip install --upgrade pip --user -q
!pip install tapipy -q
from tapipy.tapis import Tapis

In [3]:
# Method using getpass --- PREFERRED
# =================================

from getpass import getpass

# from tapipy.tapis import Tapis

t = Tapis(
    base_url="https://designsafe.tapis.io",
    username=getpass("input username: "),
    password=getpass("input password: "),
)
t.get_tokens()

input username:  ········
input password:  ········


In [4]:
# Import libraries
import os
import uuid
from datetime import date
from DS_GenFunctionsV3 import *  # General python functions useful for DesignSafe

# ---------------------------------------------------------------------------------
# Identify folder with input file in DesignSafe
cur_dir = os.getcwd()
input_uri = DS_GetDir(cur_dir, t)

In [5]:
# ---------------------------------------------------------------------------------
# Define job name
jobname = "opensees-MP-multiMotion-Jupyter"

# ---------------------------------------------------------------------------------
# Select tapis-app
app_id = "opensees-mp-s3"
app_version = "3.7.0"

# ---------------------------------------------------------------------------------
# Define file to run
input_filename = "Main_multiMotion.tcl"

# ---------------------------------------------------------------------------------
# Define control tapis-app variables
control_storage_id = "designsafe.storage.default"
control_execSystem = "stampede3"  # "frontera", "stampede3-simcenter"
control_allocation = "-A DesignSafe-SimCenter"  # MUST USE YOUR OWN ALLOCATION !!
control_exec_Dir = "DS_input"  # Folder with files including input_filename
control_nodeCount = 1
control_corespernode = 16
control_memoryMB = 1000
control_maxMinutes = 60

if control_execSystem == "frontera":
    control_batchQueue = "development"  # "normal"
elif control_execSystem == "stampede3":
    control_batchQueue = "skx-dev"  # "skx"
elif control_execSystem == "stampede3-simcenter":
    control_batchQueue = "simcenter"
else:
    print("wrong execSystem")

In [6]:
# -------------------------------------------------------------------------------
# Define inputs
fileInputs = [
    {"name": "Input Directory", "sourceUrl": f"{input_uri}/{control_exec_Dir}"}
]

# -------------------------------------------------------------------------------
# Define parameters
parameterSet = {"appArgs": []}
parameterSet["appArgs"].append({"name": "Main Script", "arg": input_filename})
parameterSet["schedulerOptions"] = [{"arg": control_allocation}]

In [7]:
# -------------------------------------------------------------------------------
# Define job description
app = t.apps.getApp(appId=app_id, appVersion=app_version)
job_description = {}
job_description["name"] = jobname
job_description["appId"] = app_id
job_description["appVersion"] = app_version

job_description["execSystemLogicalQueue"] = control_batchQueue
job_description["execSystemId"] = control_execSystem
job_description["execSystemExecDir"] = "${JobWorkingDir}"

job_description["archiveSystemId"] = control_storage_id
job_description[
    "archiveSystemDir"
] = "${EffectiveUserId}/tapis-jobs-archive/${JobCreateDate}/${JobUUID}"

job_description["maxMinutes"] = control_maxMinutes
job_description["nodeCount"] = control_nodeCount
job_description["coresPerNode"] = control_corespernode

job_description["fileInputs"] = fileInputs
job_description["parameterSet"] = parameterSet

### Run job

In [8]:
# Submit job
job = t.jobs.submitJob(**job_description)
jobUuid = job.uuid

print(" Job launched. Status provided below")
print(
    " Can also check in DedignSafe portal under - Workspace > Tools & Application > Job Status"
)

 Job launched. Status provided below
 Can also check in DedignSafe portal under - Workspace > Tools & Application > Job Status


In [9]:
# Check status
status = DS_GetStatus(t, jobUuid, tlapse=1)

# Check Runtime
DS_GetRuntime(t, jobUuid)

 Job launched. Status provided below
 Can also check in DesignSafe portal under - Workspace > Tools & Application > Job Status
	Status: PENDING
	Status: PROCESSING_INPUTS
	Status: STAGING_INPUTS
	Status: STAGING_JOB
	Status: SUBMITTING_JOB
	Status: FAILED

Runtime Summary
---------------
TOTAL   time: 0:01:11.880341


# Postprocess Results

### Identify job, archived location and user

In [10]:
jobinfo = t.jobs.getJob(jobUuid=job.uuid)
jobinfo.archiveSystemDir
user = jobinfo.createdby

### Go to archived folder

In [11]:
os.chdir(jobinfo.archiveSystemDir.replace(user, "/home/jupyter/MyData"))
# os.chdir(control_exec_Dir)
os.chdir("inputDirectory")

FileNotFoundError: [Errno 2] No such file or directory: 'inputDirectory'

### Import python libraries

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

### Plot acceleration response spectra

Plot acceleration response spectra on log-linear scale

In [ ]:
from plotAcc import plot_acc

plot_acc()